In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import warnings 
import os 
import tqdm 
import random 
from keras.preprocessing.image import load_img
input_path = []  #creating 2 lists                                               
label = []   
for class_name in os.listdir("/content/drive/MyDrive/Tree-noTree"):  #iterating directories                  
                             #class_name variable contains 2 values tree & no tree          
    for path in os.listdir("/content/drive/MyDrive/Tree-noTree/"+class_name): #iterarte each folder             
                                                                                
        if class_name == 'tree-1':
            label.append(1)
        else: 
            label.append(0)
        input_path.append(os.path.join("/content/drive/MyDrive/Tree-noTree", class_name, path))
df = pd.DataFrame()
df['images'] = input_path
df['label'] = label  
#shuffle data 
df = df.sample(frac=1).reset_index(drop=True)  
df['label'] = df['label'].astype('str')
from sklearn.model_selection import train_test_split  
train, test = train_test_split(df, test_size=0.2, random_state=42)  
from keras.preprocessing.image import ImageDataGenerator                                                                    
train_generator = ImageDataGenerator(                                                                                       
    rescale = 1.0/255,  # normalization of images                                                                              
    rotation_range = 40, # augmention of images to avoid overfitting                                                                             
    shear_range = 0.2, 
    zoom_range = 0.2,
    horizontal_flip = True, 
    fill_mode = 'nearest'
) 
val_generator = ImageDataGenerator(rescale = 1.0/255) #for test data
train_iterator = train_generator.flow_from_dataframe(
    train, 
    x_col='images', 
    y_col='label', 
    target_size=(128,128), 
    batch_size=512, 
    class_mode='binary'
)
val_iterator = val_generator.flow_from_dataframe( 
    test, 
    x_col='images', 
    y_col='label', 
    target_size=(128,128), 
    batch_size=512, 
    class_mode='binary'
)
from keras import Sequential 
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense 

model = Sequential([  #arrange  layers in sequencial manner
                    Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)), #here 16 are (hidden units) no of kernels/featureDetector/Filter
                                                        #3X3 is kernel size->filterMap//Resolution of image(128,128)*rbg(3)
                    MaxPool2D((2,2)),
                    Conv2D(32, (3,3), activation='relu'),  #for another layer
                    MaxPool2D((2,2)),
                    Conv2D(64, (3,3), activation='relu'),
                    MaxPool2D((2,2)),
                    Flatten(),
                    Dense(512, activation='relu'),   #512 is no. of nuerons
                    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_iterator, epochs=1, validation_data=val_iterator) 
model.evaluate(val_iterator) 

Found 800 validated image filenames belonging to 2 classes.
Found 200 validated image filenames belonging to 2 classes.
1/1 [==============================] - 7s 7s/step - loss: 0.6915 - accuracy: 0.4500


[0.6914700865745544, 0.44999998807907104]

In [ ]:
import PIL 
red_image = PIL.Image.open("/content/drive/MyDrive/Tree-noTree/no-tree-0/FJI_AC_1004.png")
red_image_rgb = red_image.convert("RGB")
rgb_pixel_value = red_image_rgb.getpixel((10,15))
print(rgb_pixel_value)

(0, 0, 0)


In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')
model.save_weights('try.h5')

In [ ]:
from keras.preprocessing import image
img_pred=image.load_img('/content/drive/MyDrive/Tree-noTree/tree-1/FJI_AP_102.png',target_size=(128,128))
img_pred=image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred,axis=0)

result = model.predict(img_pred)
print (result)
if abs(result)==1: 
    print("tree") 
else:
    print("no tree")

[[1.]]
tree


In [ ]:
# another method to write
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
def prediction(path):
  img=load_img(path,target_size=(224,224)) # load image
  i=img_to_array(img) # convert image to array 
  i=preprocess_input(i) 
  input_arr=np.array([i]) # increase dimensions
  print(input_arr.shape)  
  pred=np.argmax(model.predict(input_arr))
  if pred==0:
    print("The image is not of a Tree")
  else:
    print("The image is of a Tree") 
  plt.imshow(input_arr[0])
  plt.title("input image")
  plt.axis=False
  plt.show()
  print(pred)

In [ ]:
# iterate in folder to find images
from os import listdir
from os .path import isfile, join
from keras.preprocessing.image import load_img
pred_dir_path = 'link'
onlyfiles = [f for f in listdir(pred_dir_path) if isfile(join(pred_dir_path))]
print(onlyfiles) 

In [ ]:
from keras.preprocessing import image
tree_counter=0
no_tree_counter=0
for files in onlyfiles: # files jo upr vale se output aega 
  img = image.load_img(pred_dir_path+file, target_size=(128,128))
  img = image.img_to_array(img)

# ... same as above

.



















DONE



















.

"""
Script for predicting label as
good or bad given the image
It preprocess the image same as it 
was pre-processed in training 
"""
import argparse
from sklearn.externals import joblib
import numpy as np
import scipy.misc as ms
import scipy.ndimage as nd
from utils import (resize, validate, path_validation)
from config import (MODEL_PATH, WIDTH, HEIGHT,
                    RADIUS)

def predict_preprocess(IMAGE_PATH):
    """
    Preprocess the image given path of the image/images
    to apply median filter and resizing the image
    same as done in training the network
    @ Parameters:
    -------------
    IMAGE_PATH: str
        Path of the images
    @ Returns:
    ----------
    img: np.array
        filtered and pre-processed combined
        images arrays 
    
    """
    # Reading images in grayscale mode
    img = ms.imread(IMAGE_PATH, mode='L')
    # APllying median filter to remove noise
    img = nd.median_filter(img, RADIUS)
    # To make it 2D
    img = img[np.newaxis, :]
    # Resizing the images to that of train
    img = resize(img, width=WIDTH, height=HEIGHT)
    # Addition of bias term
    img = np.insert(img, 0, 1, axis=1)

    return img


def main():
    """
    Parse the argument and check validaton
    of passed image and trained model path
    Predict the label of images passed after 
    pre-process the images same as done in 
    training part
    
    """
    # Construct the argument parser and parse the arguments
    ap = argparse.ArgumentParser()
    ap.add_argument("-img", "--image_path", required=True, 
                        help="path to image")
    args = vars(ap.parse_args())

    IMAGE_PATH = args["image_path"]

    # Path Validation of image and Model
    if not path_validation(IMAGE_PATH, read_access=True):
        exit(0)
    if not path_validation(MODEL_PATH, read_access=True):
        exit(0)

    # Preprocessed the images
    img = predict_preprocess(IMAGE_PATH)

    # Load the trained NN model
    params = joblib.load(MODEL_PATH)

    # Find the label predicted by the model
    predicted_label = validate(params, img)

    for label in predicted_label:
        if label:
            print("Good Image\n")
        else:
            print("Bad Image\n")


if __name__ == "__main__":
    main()